In [1]:
import popular
import unpopular
import numpy as np
import polars as pl
import lightkurve as lk

from tqdm import tqdm
from scipy.signal import find_peaks
from tesscube import TESSCube
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from concurrent.futures.thread import ThreadPoolExecutor

In [2]:
def _count_harmonics(
    lc: lk.LightCurve, height: float = 0.15
) -> list[tuple[float, float]]:
    """Find the harmonics in the L-S periodogram of a given lightcurve.

    Args:
        lc (lk.LightCurve)
        height (float, optional): The minimum height of a peak as a fraction of the main harmonic. Defaults to 0.15.

    Returns:
        list[tuple[float, float]]: A list containing each harmonic as a tuple of period and power.
    """

    pg = lc.to_periodogram()
    period = pg.period_at_max_power

    if period.value >= 2:
        return []

    expected_harmonics = []
    for i in range(1, 9):
        expected_harmonics.append(period.value / i)

    peaks, properties = find_peaks(
        pg.power, distance=120, height=pg.max_power.value * height
    )

    peak_periods = [pg.period[idx].value for idx in peaks]

    found_harmonics = []
    for i, period in enumerate(peak_periods):
        in_range = 0.9 * expected_harmonics[i] <= period <= 1.1 * expected_harmonics[i]
        if in_range:
            found_harmonics.append((period, properties["peak_heights"][i]))

    return found_harmonics

def is_complex(lc: lk.LightCurve) -> bool:
    """Check if a given lightcurve is complex by counting the number of harmonics."""
    return len(_count_harmonics(lc)) >= 3

In [3]:
def get_tpf(coords, sector, camera, ccd):
    cube = TESSCube(sector, camera, ccd)
    wcs = cube.wcs
    pixels = wcs.world_to_pixel(coords)
    
    col_pix, row_pix = np.asarray(pixels).astype(np.dtype("int"))
    corner = (row_pix - (50 // 2), col_pix - (50 // 2))
    tpf = cube.get_tpf(corner, shape=(50, 50), calculate_poscorr=False)

    return tpf


def make_lightcurve(tpf):    
    s = popular.Source(tpf, remove_bad=True)
    s.set_aperture(rowlims=[25, 26], collims=[25, 26])
    s.add_cpm_model(exclusion_size=5, n=64, predictor_method="similar_brightness")
    s.set_regs([0.1])
    s.holdout_fit_predict(k=100)
    apt_detrended_flux = s.get_aperture_lc(data_type="cpm_subtracted_flux")
    
    return lk.TessLightCurve(time=s.time, flux=apt_detrended_flux)

In [4]:
def process_target(target):
    tic, ra, dec, sector, camera, ccd = target.values()
    coords = SkyCoord(ra, dec, unit="deg", frame="icrs")

    tpf = get_tpf(coords, sector, camera, ccd)
    lc = make_lightcurve(tpf)
    return (tic, sector, is_complex(lc))

In [5]:
targets = pl.read_csv("./data/targets.csv")

In [8]:
for target in tqdm(targets[0:10].to_dicts()):
    process_target(target)

 10%|████████████████▊                                                                                                                                                       | 1/10 [00:05<00:47,  5.28s/it]

Summing over 2 x 2 pixel lightcurves. Weighting=None
Summing over 2 x 2 pixel lightcurves. Weighting=None


 20%|█████████████████████████████████▌                                                                                                                                      | 2/10 [00:20<01:30, 11.29s/it]/home/zeus/pipeline/.venv/lib/python3.12/site-packages/astropy/units/quantity.py:658: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
 30%|██████████████████████████████████████████████████▍                                                                                                                     | 3/10 [00:26<01:01,  8.73s/it]

Summing over 2 x 2 pixel lightcurves. Weighting=None
Summing over 2 x 2 pixel lightcurves. Weighting=None


 50%|████████████████████████████████████████████████████████████████████████████████████                                                                                    | 5/10 [00:46<00:43,  8.67s/it]

...

 60%|████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                   | 6/10 [00:52<00:31,  7.82s/it]

Summing over 2 x 2 pixel lightcurves. Weighting=None


 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                  | 7/10 [01:23<00:35, 11.92s/it]

Summing over 2 x 2 pixel lightcurves. Weighting=None


IndexError: list index out of range